# Cortical Surface Analysis with Spherical Harmonics

This notebook provides a complete analysis of cortical surfaces using spherical harmonics.

## Contents:
1. **Setup & Data Loading**
2. **Reconstruction Quality Analysis** 
3. **Brain Reconstruction**
4. **Coefficient Properties Analysis**

## 1. Setup & Data Loading

In [ ]:
import os
import sys
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import scipy.io as sio

# Add path for imports
sys.path.append('../')

from utils.mathutils import compute_vertex_normals, build_template_adjacency_two_hemis, compute_mean_curvature, hausdorff_distance
from utils.cortical import spherical_harmonics as SH
from utils.cortical import surface_preprocess as sp

# Set plotting style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Configuration - UPDATE THESE PATHS
# Ensure these paths are correct for your environment
DATA_PATH = r"C:\Users\wbou2\Documents\meg_to_surface_ml\src\cortical_transformation\data"
SUBJECTS_DIR = r"C:\Users\wbou2\Documents\meg_to_surface_ml\data\Anatomy_data_CAM_CAN"

# Load templates and harmonics
print("Loading templates and harmonics...")
template_lh = np.load(os.path.join(DATA_PATH, "lh_sphere_projection.npz"))
template_rh = np.load(os.path.join(DATA_PATH, "rh_sphere_projection.npz"))
Y_lh_full = np.load(os.path.join(DATA_PATH, "Y_lh.npz"))['Y']
Y_rh_full = np.load(os.path.join(DATA_PATH, "Y_rh.npz"))['Y']

# Get subject list
subjects = [f for f in os.listdir(SUBJECTS_DIR) 
           if os.path.isdir(os.path.join(SUBJECTS_DIR, f)) and f.startswith("sub-")]

print(f"Found {len(subjects)} subjects")
print(f"Harmonics shape - LH: {Y_lh_full.shape}, RH: {Y_rh_full.shape}")

## 2. Reconstruction Quality Analysis

Analyze how reconstruction quality changes with different harmonic orders.

In [ ]:
def compute_reconstruction_error(subject_path, hemi, Y_full, l, vertex_to_faces):
    """Compute reconstruction error for one subject/hemisphere"""
    # Load data
    coeffs_path = os.path.join(subject_path, f"coeffs_{hemi}.pkl")
    resampled_path = os.path.join(subject_path, f"{hemi}_resampled.npz")
    
    if not (os.path.exists(coeffs_path) and os.path.exists(resampled_path)):
        return None
        
    with open(coeffs_path, 'rb') as f:
        coeffs = pickle.load(f)
    resampled_data = np.load(resampled_path)
    
    # Truncate harmonics and coefficients
    Y_trunc = Y_full[:, :(l+1)**2]
    org_coeffs = {i: coeffs['organized_coeffs'][i] for i in range(l+1) 
                 if i in coeffs['organized_coeffs']}
    
    # Reconstruct surface
    recon_surface = SH.generate_surface(Y_trunc, l, 0, org_coeffs)
    
    # Calculate normalized Hausdorff distance
    hausdorff = hausdorff_distance(resampled_data['coords'], recon_surface)
    char_size = np.max(np.ptp(resampled_data['coords'], axis=0))
    
    return hausdorff / char_size

In [ ]:
# Analysis parameters
MAX_L = 50
STEP = 5
l_values = list(range(5, MAX_L+1, STEP))

print(f"Analyzing reconstruction quality for l values: {l_values}")

# Build adjacency once (for efficiency)
print("Building vertex adjacency...")
vertex_to_faces = build_template_adjacency_two_hemis(
    template_lh['sphere_tris'], 
    template_rh['sphere_tris']
)

# Store results
quality_metrics = {'l_values': l_values, 'lh_errors': [], 'rh_errors': []}

# Process each l value
for l in tqdm(l_values, desc="Processing l values"):
    lh_errors, rh_errors = [], []
    
    for subject in subjects:
        subject_path = os.path.join(SUBJECTS_DIR, subject)
        
        # Process both hemispheres
        lh_error = compute_reconstruction_error(subject_path, 'lh', Y_lh_full, l, vertex_to_faces)
        rh_error = compute_reconstruction_error(subject_path, 'rh', Y_rh_full, l, vertex_to_faces)
        
        if lh_error is not None:
            lh_errors.append(lh_error)
        if rh_error is not None:
            rh_errors.append(rh_error)
    
    # Store mean errors
    quality_metrics['lh_errors'].append(np.mean(lh_errors) if lh_errors else np.nan)
    quality_metrics['rh_errors'].append(np.mean(rh_errors) if rh_errors else np.nan)

print("Quality analysis completed!")

In [ ]:
# Plot reconstruction quality
plt.figure(figsize=(12, 6))

plt.plot(quality_metrics['l_values'], quality_metrics['lh_errors'], 
         'o-', label='Left Hemisphere', color='blue', linewidth=2)
plt.plot(quality_metrics['l_values'], quality_metrics['rh_errors'], 
         'o-', label='Right Hemisphere', color='red', linewidth=2)

plt.xlabel('Harmonic Order (l)', fontsize=12)
plt.ylabel('Normalized Hausdorff Distance', fontsize=12)
plt.title('Reconstruction Quality vs Harmonic Order', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.yscale('log')


plt.tight_layout()
plt.show()



## 3. Brain Reconstruction

Reconstruct complete 3D brain models and save them as MATLAB files.

In [ ]:
def reconstruct_hemisphere(hemi, l, coeffs, center, Y_full):
    """Reconstruct one hemisphere"""
    Y_trunc = Y_full[:, :(l+1)**2]
    org_coeffs = {i: coeffs['organized_coeffs'][i] for i in range(l+1)}
    coords = SH.generate_surface(Y_trunc, l, 0, org_coeffs)
    return coords + center

In [ ]:
# Reconstruction parameters
RECONSTRUCTION_L = 30  # Use the optimal l or adjust as needed

print(f"Reconstructing all brains with l={RECONSTRUCTION_L}...")

# Get triangulation from first subject
first_subject_path = os.path.join(SUBJECTS_DIR, subjects[0])
tris = np.load(os.path.join(first_subject_path, "lh_resampled.npz"))['tris']

# Reconstruct each subject
reconstruction_results = []

for subject in tqdm(subjects, desc="Reconstructing brains"):
    subject_path = os.path.join(SUBJECTS_DIR, subject)
    
    try:
        # Load centers and coefficients
        lh_data = np.load(os.path.join(subject_path, "lh_resampled.npz"))
        rh_data = np.load(os.path.join(subject_path, "rh_resampled.npz"))
        
        with open(os.path.join(subject_path, "coeffs_lh.pkl"), 'rb') as f:
            coeffs_lh = pickle.load(f)
        with open(os.path.join(subject_path, "coeffs_rh.pkl"), 'rb') as f:
            coeffs_rh = pickle.load(f)
        
        # Reconstruct hemispheres
        lh_coords = reconstruct_hemisphere('lh', RECONSTRUCTION_L, coeffs_lh, lh_data['center'], Y_lh_full)
        rh_coords = reconstruct_hemisphere('rh', RECONSTRUCTION_L, coeffs_rh, rh_data['center'], Y_rh_full)
        
        # Merge hemispheres
        merged_coords, merged_tris = sp.merge_hemis((lh_coords, tris), (rh_coords, tris))
        
        # Save as .mat file for MATLAB
        TessMat = {
            'Vertices': merged_coords,
            'Faces': merged_tris + 1  # MATLAB uses 1-based indexing
        }
        mat_path = os.path.join(subject_path, 'brain_reconstructed.mat')
        sio.savemat(mat_path, {'TessMat': TessMat})
        
        reconstruction_results.append({
            'subject': subject,
            'n_vertices': len(merged_coords),
            'n_faces': len(merged_tris),
            'success': True
        })
        
    except Exception as e:
        print(f"Error reconstructing {subject}: {e}")
        reconstruction_results.append({
            'subject': subject,
            'success': False,
            'error': str(e)
        })

# Summary
successful = sum(1 for r in reconstruction_results if r['success'])
print(f"\nReconstruction completed: {successful}/{len(subjects)} subjects successful")

if successful > 0:
    example_result = next(r for r in reconstruction_results if r['success'])
    print(f"Example: {example_result['n_vertices']} vertices, {example_result['n_faces']} faces per brain")

## 4. Coefficient Properties Analysis

Analyze the properties of spherical harmonic coefficients.

In [ ]:
# Load all coefficients
print("Loading all coefficients...")
all_coeffs = {'lh': [], 'rh': []}

for subject in tqdm(subjects, desc="Loading coefficients"):
    subject_path = os.path.join(SUBJECTS_DIR, subject)
    
    try:
        for hemi in ['lh', 'rh']:
            with open(os.path.join(subject_path, f"coeffs_{hemi}.pkl"), 'rb') as f:
                coeffs = pickle.load(f)
            all_coeffs[hemi].append(coeffs['organized_coeffs'])
    except Exception as e:
        print(f"Error loading coefficients for {subject}: {e}")

print(f"Loaded coefficients for {len(all_coeffs['lh'])} subjects")

In [ ]:
# Analyze spectral power decay
def compute_spectral_power(coeffs_list, max_l=50):
    """Compute spectral power for each harmonic order"""
    powers = []
    
    for subject_coeffs in coeffs_list:
        subject_powers = []
        for l in range(1, min(max_l+1, len(subject_coeffs))):
            if l in subject_coeffs:
                # Calculate power for this order (sum over all coordinates)
                power = np.sum(np.abs(subject_coeffs[l])**2)
                subject_powers.append(power)
        powers.append(subject_powers)
    
    return np.array(powers)

# Compute powers
lh_powers = compute_spectral_power(all_coeffs['lh'])
rh_powers = compute_spectral_power(all_coeffs['rh'])

# Plot spectral power decay
plt.figure(figsize=(12, 6))

l_range = range(1, lh_powers.shape[1]+1)
lh_mean = np.mean(lh_powers, axis=0)
rh_mean = np.mean(rh_powers, axis=0)

plt.semilogy(l_range, lh_mean, 'o-', color='blue', label='Left Hemisphere', linewidth=2)
plt.semilogy(l_range, rh_mean, 'o-', color='red', label='Right Hemisphere', linewidth=2)

plt.xlabel('Harmonic Order (l)', fontsize=12)
plt.ylabel('Spectral Power (log scale)', fontsize=12)
plt.title('Spectral Power Decay Across Harmonic Orders', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

# Print power decay rate
decay_rate_lh = np.polyfit(np.log(l_range[10:30]), np.log(lh_mean[10:30]), 1)[0]
decay_rate_rh = np.polyfit(np.log(l_range[10:30]), np.log(rh_mean[10:30]), 1)[0]
print(f"Power decay rates (l=10-30): LH={decay_rate_lh:.2f}, RH={decay_rate_rh:.2f}")

In [ ]:
# Analyze coefficient distributions for specific orders
analysis_orders = [5, 15, 30]

fig, axes = plt.subplots(len(analysis_orders), 2, figsize=(12, 4*len(analysis_orders)))
if len(analysis_orders) == 1:
    axes = axes.reshape(1, -1)

for i, l in enumerate(analysis_orders):
    for j, (hemi, color) in enumerate([('lh', 'blue'), ('rh', 'red')]):
        # Collect all coefficients for this l and hemisphere
        coeffs_flat = []
        for subject_coeffs in all_coeffs[hemi]:
            if l < len(subject_coeffs) and l in subject_coeffs:
                coeffs_flat.extend(np.real(subject_coeffs[l].flatten()))
        
        # Plot histogram and statistics
        axes[i, j].hist(coeffs_flat, bins=50, alpha=0.7, color=color, density=True)
        axes[i, j].set_title(f'{hemi.upper()} - Order l={l}\n(μ={np.mean(coeffs_flat):.3f}, σ={np.std(coeffs_flat):.3f})')
        axes[i, j].set_xlabel('Coefficient Value')
        axes[i, j].set_ylabel('Density')
        axes[i, j].grid(True, alpha=0.3)
        
        # Overlay normal distribution
        x = np.linspace(min(coeffs_flat), max(coeffs_flat), 100)
        normal_fit = (1/np.sqrt(2*np.pi*np.var(coeffs_flat))) * np.exp(-0.5*(x-np.mean(coeffs_flat))**2/np.var(coeffs_flat))
        axes[i, j].plot(x, normal_fit, 'k--', alpha=0.8, label='Normal fit')
        axes[i, j].legend()

plt.tight_layout()
plt.show()

## Summary

This notebook provided a complete analysis of cortical surfaces using spherical harmonics:

1. **Quality Analysis**: Showed how reconstruction error decreases with higher harmonic orders
2. **Brain Reconstruction**: Generated complete 3D brain models from harmonic coefficients
3. **Coefficient Properties**: Analyzed spectral power decay and coefficient distributions

### Key Findings:
- Optimal harmonic order for reconstruction quality
- Power law decay of spectral energy
- Gaussian-like distribution of coefficients

### Output Files:
- `brain_reconstructed.mat`: Complete 3D brain models for each subject